#**Pré-processamento** 

In [ ]:
from google.colab import drive 
drive.mount('/content/gdrive')

import pandas as pd
import seaborn as sns
import missingno as msno
import warnings
from matplotlib import pyplot as plt
import datetime
import numpy as np
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf
from matplotlib import pyplot
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns',100)

df = pd.read_csv("gdrive/My Drive/dissertação/Qualidade_do_ar_-_Dados_horarios.csv",sep=",")
df['Data']=df['Data'].astype('str')
df['Data']=df['Data'].apply(lambda x: x[:-3])

#conversão para datetime
def convert_to_datetime(x):
    return(datetime.datetime.strptime(x, "%Y/%m/%d %H:%M:%S"))
df['Data']=df['Data'].apply(convert_to_datetime) 
df['Hora-minuto']=df['Data'].apply(lambda x: str(x)[-8:-3])
# #extrair ano, mes, hora 
df['Ano']=df['Data'].apply(lambda x: x.year)
df['Mês']=df['Data'].apply(lambda x: x.month)
df['Dia']=df['Data'].apply(lambda x: x.day)
df['Hora']=df['Data'].apply(lambda x: x.hour)
df['Hora-minuto']=df['Data'].apply(lambda x: str(x)[-11:-6])

df_bg=df[df['Estação']=='BG'] #Bangu
df_gc=df[df['Estação']=='CG'] #Campo Grande
df_ca=df[df['Estação']=='CA'] #Centro
df_av=df[df['Estação']=='AV'] #Copacabana
df_ir=df[df['Estação']=='IR'] #Irajá
df_pg=df[df['Estação']=='PG'] #Pedra de Guaratiba
df_sc=df[df['Estação']=='SC'] #São Cristóvão
df_sp=df[df['Estação']=='SP'] #Tijuca

df_sc.set_index('Data',inplace=True)
df_sc.drop(columns=['OBJECTID','CodNum','Estação','Dir_Vento','NO2','HCNM','HCT','CH4','NO','NOx',\
                   'PM2_5', 'Lat', 'Lon', 'X_UTM_Sirgas2000','Y_UTM_Sirgas2000'], inplace=True)
df_sc_2=df_sc.copy() #para fazer distribuição sem eliminar ausentes

df_sc=df_sc['2011-01-01':'2018-01-01']
#explicar que usou interpolação linear com spline e justificar
df_sc['Chuva'].interpolate(method='slinear', inplace=True)
df_sc['Pres'].interpolate(method='slinear', inplace=True)
df_sc['RS'].interpolate(method='slinear', inplace=True)
df_sc['Temp'].interpolate(method='slinear', inplace=True)
df_sc['UR'].interpolate(method='slinear', inplace=True)
df_sc['Vel_Vento'].interpolate(method='slinear', inplace=True)
df_sc['SO2'].interpolate(method='slinear', inplace=True)
df_sc['CO'].interpolate(method='slinear', inplace=True)
df_sc['O3'].interpolate(method='slinear', inplace=True)
df_sc['PM10'].interpolate(method='slinear', inplace=True)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


#**(spline) janela=1**

In [ ]:
# prepare data for lstm
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score #colocar citação

from math import sqrt
from numpy import concatenate
from matplotlib import pyplot

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.optimizers import Adam

df_sc.drop(columns=['Ano','Mês','Dia','Hora','Hora-minuto'], inplace=True)
#df_sc.drop(columns=['Hora-minuto'], inplace=True)
df_sc_copy=df_sc.copy()
df_sc_copy=df_sc_copy[['O3']]
df_sc.drop(columns=['O3'],inplace=True)
df_sc = pd.concat([df_sc_copy,df_sc], axis=1, ignore_index=False)

values = df_sc.values
# ensure all data is float
values = values.astype('float32')
# normalize features

# scaler = MinMaxScaler(feature_range=(0, 1))
# scaled = scaler.fit_transform(values)

scaled=values #não normalizado

# convert series to supervised learning
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
	n_vars = 1 if type(data) is list else data.shape[1]
	df = DataFrame(data)
	cols, names = list(), list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
		names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
		if i == 0:
			names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
		else:
			names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
	# put it all together
	agg = concat(cols, axis=1)
	agg.columns = names
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg

# specify the number of lag hours
n_hours = 1 #aqui definimos o LAg
n_features = 10

# frame as supervised learning
# reframed = series_to_supervised(scaled, n_hours, 1)
reframed = series_to_supervised(scaled, n_hours, 1) #é aqui que definimos a janela de previsão

# split into train and test sets
values = reframed.values
n_train_hours = 1855 * 24
train = values[:n_train_hours, :]
test = values[n_train_hours:, :]

# split into input and outputs
n_obs = n_hours * n_features
train_X, train_y = train[:, :n_obs], train[:, -n_features]
test_X, test_y = test[:, :n_obs], test[:, -n_features]

# reshape input to be 3D [samples, timesteps, features]
train_X = train_X.reshape((train_X.shape[0], n_hours, n_features))
test_X = test_X.reshape((test_X.shape[0], n_hours, n_features))

# design network
opt = Adam(learning_rate=0.0005)
model = Sequential()
model.add(LSTM(300, input_shape=(train_X.shape[1], train_X.shape[2])))
model.add(Dense(1))
model.compile(loss='mae', optimizer=opt)

# fit network
history = model.fit(train_X, train_y, epochs=150, batch_size=500, validation_data=(test_X, test_y), verbose=2, shuffle=False)

Epoch 1/150
90/90 - 1s - loss: 18.5955 - val_loss: 17.7760
Epoch 2/150
90/90 - 0s - loss: 16.5612 - val_loss: 16.1010
Epoch 3/150
90/90 - 0s - loss: 15.0051 - val_loss: 14.6024
Epoch 4/150
90/90 - 0s - loss: 13.6808 - val_loss: 13.4055
Epoch 5/150
90/90 - 0s - loss: 12.6145 - val_loss: 12.4107
Epoch 6/150
90/90 - 0s - loss: 11.7774 - val_loss: 11.6263
Epoch 7/150
90/90 - 0s - loss: 11.0767 - val_loss: 11.0414
Epoch 8/150
90/90 - 0s - loss: 10.5449 - val_loss: 10.5041
Epoch 9/150
90/90 - 0s - loss: 9.9712 - val_loss: 10.0051
Epoch 10/150
90/90 - 0s - loss: 9.5619 - val_loss: 9.5837
Epoch 11/150
90/90 - 0s - loss: 9.2285 - val_loss: 9.2092
Epoch 12/150
90/90 - 0s - loss: 8.8958 - val_loss: 8.9227
Epoch 13/150
90/90 - 0s - loss: 8.6175 - val_loss: 8.6416
Epoch 14/150
90/90 - 0s - loss: 8.3549 - val_loss: 8.4827
Epoch 15/150
90/90 - 0s - loss: 8.1797 - val_loss: 8.1316
Epoch 16/150
90/90 - 0s - loss: 7.9769 - val_loss: 8.0349
Epoch 17/150
90/90 - 0s - loss: 7.8754 - val_loss: 7.8596
Epoch 

In [ ]:
# make a prediction
yhat = model.predict(test_X)
test_X = test_X.reshape((test_X.shape[0], n_hours*n_features))

# invert scaling for forecast (não foi feito)
inv_yhat = concatenate((yhat, test_X[:, -9:]), axis=1)
inv_yhat = inv_yhat[:,0]

# invert scaling for actual
test_y = test_y.reshape((len(test_y), 1))
inv_y = concatenate((test_y, test_X[:, -9:]), axis=1)
# inv_y = scaler.inverse_transform(inv_y)
inv_y = inv_y[:,0]

# calculate RMSE
rmse = sqrt(mean_squared_error(inv_y, inv_yhat))
print('Test RMSE: %.3f' % rmse)

Test RMSE: 9.789


In [ ]:
# calculate MAE
mae = mean_absolute_error(inv_y, inv_yhat)
print('Test MAE: %.3f' % mae)

Test MAE: 6.054


In [ ]:
# calculate R2
r2=r2_score(inv_y, inv_yhat)
print('Test R2: %.3f' % r2)

Test R2: 0.832


#**(spline) janela=2**

In [ ]:
# prepare data for lstm
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score #colocar citação

from math import sqrt
from numpy import concatenate
from matplotlib import pyplot

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.optimizers import Adam

df_sc.drop(columns=['Ano','Mês','Dia','Hora','Hora-minuto'], inplace=True)
#df_sc.drop(columns=['Hora-minuto'], inplace=True)
df_sc_copy=df_sc.copy()
df_sc_copy=df_sc_copy[['O3']]
df_sc.drop(columns=['O3'],inplace=True)
df_sc = pd.concat([df_sc_copy,df_sc], axis=1, ignore_index=False)

values = df_sc.values
# ensure all data is float
values = values.astype('float32')
# normalize features

# scaler = MinMaxScaler(feature_range=(0, 1))
# scaled = scaler.fit_transform(values)

scaled=values #não normalizado

# convert series to supervised learning
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
	n_vars = 1 if type(data) is list else data.shape[1]
	df = DataFrame(data)
	cols, names = list(), list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
		names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
		if i == 0:
			names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
		else:
			names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
	# put it all together
	agg = concat(cols, axis=1)
	agg.columns = names
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg

# specify the number of lag hours
n_hours = 2 #aqui definimos o LAg
n_features = 10

# frame as supervised learning
# reframed = series_to_supervised(scaled, n_hours, 1)
reframed = series_to_supervised(scaled, n_hours, 2) #é aqui que definimos a janela de previsão

# split into train and test sets
values = reframed.values
n_train_hours = 1855 * 24
train = values[:n_train_hours, :]
test = values[n_train_hours:, :]

# split into input and outputs
n_obs = n_hours * n_features
train_X, train_y = train[:, :n_obs], train[:, -n_features]
test_X, test_y = test[:, :n_obs], test[:, -n_features]

# reshape input to be 3D [samples, timesteps, features]
train_X = train_X.reshape((train_X.shape[0], n_hours, n_features))
test_X = test_X.reshape((test_X.shape[0], n_hours, n_features))

# design network
opt = Adam(learning_rate=0.0005)
model = Sequential()
model.add(LSTM(300, input_shape=(train_X.shape[1], train_X.shape[2])))
model.add(Dense(1))
model.compile(loss='mae', optimizer=opt)

# fit network
history = model.fit(train_X, train_y, epochs=150, batch_size=500, validation_data=(test_X, test_y), verbose=2, shuffle=False)

Epoch 1/150
90/90 - 1s - loss: 18.6968 - val_loss: 17.9202
Epoch 2/150
90/90 - 0s - loss: 16.7208 - val_loss: 16.3746
Epoch 3/150
90/90 - 0s - loss: 15.4136 - val_loss: 15.1727
Epoch 4/150
90/90 - 0s - loss: 14.4315 - val_loss: 14.2557
Epoch 5/150
90/90 - 0s - loss: 13.6249 - val_loss: 13.4887
Epoch 6/150
90/90 - 0s - loss: 13.0052 - val_loss: 12.8662
Epoch 7/150
90/90 - 0s - loss: 12.4696 - val_loss: 12.3487
Epoch 8/150
90/90 - 0s - loss: 11.9592 - val_loss: 11.9347
Epoch 9/150
90/90 - 0s - loss: 11.5718 - val_loss: 11.5834
Epoch 10/150
90/90 - 0s - loss: 11.2295 - val_loss: 11.2134
Epoch 11/150
90/90 - 0s - loss: 10.9525 - val_loss: 10.9744
Epoch 12/150
90/90 - 0s - loss: 10.6917 - val_loss: 10.7059
Epoch 13/150
90/90 - 0s - loss: 10.4946 - val_loss: 10.4990
Epoch 14/150
90/90 - 0s - loss: 10.2741 - val_loss: 10.2697
Epoch 15/150
90/90 - 0s - loss: 10.0650 - val_loss: 10.1189
Epoch 16/150
90/90 - 0s - loss: 9.9165 - val_loss: 9.9771
Epoch 17/150
90/90 - 0s - loss: 9.7978 - val_loss: 

In [ ]:
# make a prediction
yhat = model.predict(test_X)
test_X = test_X.reshape((test_X.shape[0], n_hours*n_features))

# invert scaling for forecast (não foi feito)
inv_yhat = concatenate((yhat, test_X[:, -9:]), axis=1)
inv_yhat = inv_yhat[:,0]

# invert scaling for actual
test_y = test_y.reshape((len(test_y), 1))
inv_y = concatenate((test_y, test_X[:, -9:]), axis=1)
# inv_y = scaler.inverse_transform(inv_y)
inv_y = inv_y[:,0]

# calculate RMSE
rmse = sqrt(mean_squared_error(inv_y, inv_yhat))
print('Test RMSE: %.3f' % rmse)

Test RMSE: 13.288


In [ ]:
# calculate MAE
mae = mean_absolute_error(inv_y, inv_yhat)
print('Test MAE: %.3f' % mae)

Test MAE: 8.455


In [ ]:
# calculate R2
r2=r2_score(inv_y, inv_yhat)
print('Test R2: %.3f' % r2)

Test R2: 0.691


#**(spline) janela=3**

In [ ]:
# prepare data for lstm
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score #colocar citação

from math import sqrt
from numpy import concatenate
from matplotlib import pyplot

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.optimizers import Adam

df_sc.drop(columns=['Ano','Mês','Dia','Hora','Hora-minuto'], inplace=True)
#df_sc.drop(columns=['Hora-minuto'], inplace=True)
df_sc_copy=df_sc.copy()
df_sc_copy=df_sc_copy[['O3']]
df_sc.drop(columns=['O3'],inplace=True)
df_sc = pd.concat([df_sc_copy,df_sc], axis=1, ignore_index=False)

values = df_sc.values
# ensure all data is float
values = values.astype('float32')
# normalize features

# scaler = MinMaxScaler(feature_range=(0, 1))
# scaled = scaler.fit_transform(values)

scaled=values #não normalizado

# convert series to supervised learning
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
	n_vars = 1 if type(data) is list else data.shape[1]
	df = DataFrame(data)
	cols, names = list(), list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
		names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
		if i == 0:
			names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
		else:
			names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
	# put it all together
	agg = concat(cols, axis=1)
	agg.columns = names
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg

# specify the number of lag hours
n_hours = 3 #aqui definimos o LAg
n_features = 10

# frame as supervised learning
# reframed = series_to_supervised(scaled, n_hours, 1)
reframed = series_to_supervised(scaled, n_hours, 3) #é aqui que definimos a janela de previsão

# split into train and test sets
values = reframed.values
n_train_hours = 1855 * 24
train = values[:n_train_hours, :]
test = values[n_train_hours:, :]

# split into input and outputs
n_obs = n_hours * n_features
train_X, train_y = train[:, :n_obs], train[:, -n_features]
test_X, test_y = test[:, :n_obs], test[:, -n_features]

# reshape input to be 3D [samples, timesteps, features]
train_X = train_X.reshape((train_X.shape[0], n_hours, n_features))
test_X = test_X.reshape((test_X.shape[0], n_hours, n_features))

# design network
opt = Adam(learning_rate=0.0005)
model = Sequential()
model.add(LSTM(300, input_shape=(train_X.shape[1], train_X.shape[2])))
model.add(Dense(1))
model.compile(loss='mae', optimizer=opt)

# fit network
history = model.fit(train_X, train_y, epochs=150, batch_size=500, validation_data=(test_X, test_y), verbose=2, shuffle=False)

Epoch 1/150
90/90 - 1s - loss: 18.7842 - val_loss: 18.0098
Epoch 2/150
90/90 - 0s - loss: 17.1761 - val_loss: 17.0412
Epoch 3/150
90/90 - 0s - loss: 16.3739 - val_loss: 16.2493
Epoch 4/150
90/90 - 0s - loss: 15.6419 - val_loss: 15.5124
Epoch 5/150
90/90 - 0s - loss: 14.9723 - val_loss: 14.9108
Epoch 6/150
90/90 - 0s - loss: 14.4408 - val_loss: 14.4085
Epoch 7/150
90/90 - 0s - loss: 13.9676 - val_loss: 13.9757
Epoch 8/150
90/90 - 0s - loss: 13.5797 - val_loss: 13.6224
Epoch 9/150
90/90 - 0s - loss: 13.2677 - val_loss: 13.3197
Epoch 10/150
90/90 - 0s - loss: 12.9293 - val_loss: 12.9635
Epoch 11/150
90/90 - 0s - loss: 12.6362 - val_loss: 12.7464
Epoch 12/150
90/90 - 0s - loss: 12.5126 - val_loss: 12.5398
Epoch 13/150
90/90 - 0s - loss: 12.2700 - val_loss: 12.3906
Epoch 14/150
90/90 - 0s - loss: 12.1425 - val_loss: 12.2583
Epoch 15/150
90/90 - 0s - loss: 11.9738 - val_loss: 12.0650
Epoch 16/150
90/90 - 0s - loss: 11.8195 - val_loss: 11.9716
Epoch 17/150
90/90 - 0s - loss: 11.7300 - val_los

In [ ]:
# make a prediction
yhat = model.predict(test_X)
test_X = test_X.reshape((test_X.shape[0], n_hours*n_features))

# invert scaling for forecast (não foi feito)
inv_yhat = concatenate((yhat, test_X[:, -9:]), axis=1)
inv_yhat = inv_yhat[:,0]

# invert scaling for actual
test_y = test_y.reshape((len(test_y), 1))
inv_y = concatenate((test_y, test_X[:, -9:]), axis=1)
# inv_y = scaler.inverse_transform(inv_y)
inv_y = inv_y[:,0]

# calculate RMSE
rmse = sqrt(mean_squared_error(inv_y, inv_yhat))
print('Test RMSE: %.3f' % rmse)

Test RMSE: 15.545


In [ ]:
# calculate MAE
mae = mean_absolute_error(inv_y, inv_yhat)
print('Test MAE: %.3f' % mae)

Test MAE: 10.219


In [ ]:
# calculate R2
r2=r2_score(inv_y, inv_yhat)
print('Test R2: %.3f' % r2)

Test R2: 0.577


#**(spline) janela=4**

In [ ]:
# prepare data for lstm
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score #colocar citação

from math import sqrt
from numpy import concatenate
from matplotlib import pyplot

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.optimizers import Adam

df_sc.drop(columns=['Ano','Mês','Dia','Hora','Hora-minuto'], inplace=True)
#df_sc.drop(columns=['Hora-minuto'], inplace=True)
df_sc_copy=df_sc.copy()
df_sc_copy=df_sc_copy[['O3']]
df_sc.drop(columns=['O3'],inplace=True)
df_sc = pd.concat([df_sc_copy,df_sc], axis=1, ignore_index=False)

values = df_sc.values
# ensure all data is float
values = values.astype('float32')
# normalize features

# scaler = MinMaxScaler(feature_range=(0, 1))
# scaled = scaler.fit_transform(values)

scaled=values #não normalizado

# convert series to supervised learning
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
	n_vars = 1 if type(data) is list else data.shape[1]
	df = DataFrame(data)
	cols, names = list(), list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
		names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
		if i == 0:
			names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
		else:
			names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
	# put it all together
	agg = concat(cols, axis=1)
	agg.columns = names
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg

# specify the number of lag hours
n_hours = 4 #aqui definimos o LAg
n_features = 10

# frame as supervised learning
# reframed = series_to_supervised(scaled, n_hours, 1)
reframed = series_to_supervised(scaled, n_hours, 4) #é aqui que definimos a janela de previsão

# split into train and test sets
values = reframed.values
n_train_hours = 1855 * 24
train = values[:n_train_hours, :]
test = values[n_train_hours:, :]

# split into input and outputs
n_obs = n_hours * n_features
train_X, train_y = train[:, :n_obs], train[:, -n_features]
test_X, test_y = test[:, :n_obs], test[:, -n_features]

# reshape input to be 3D [samples, timesteps, features]
train_X = train_X.reshape((train_X.shape[0], n_hours, n_features))
test_X = test_X.reshape((test_X.shape[0], n_hours, n_features))

# design network
opt = Adam(learning_rate=0.0005)
model = Sequential()
model.add(LSTM(300, input_shape=(train_X.shape[1], train_X.shape[2])))
model.add(Dense(1))
model.compile(loss='mae', optimizer=opt)

# fit network
history = model.fit(train_X, train_y, epochs=150, batch_size=500, validation_data=(test_X, test_y), verbose=2, shuffle=False)

Epoch 1/150
90/90 - 1s - loss: 18.8333 - val_loss: 18.2134
Epoch 2/150
90/90 - 0s - loss: 17.4647 - val_loss: 17.3999
Epoch 3/150
90/90 - 0s - loss: 16.8039 - val_loss: 16.7524
Epoch 4/150
90/90 - 0s - loss: 16.2952 - val_loss: 16.2402
Epoch 5/150
90/90 - 0s - loss: 15.8390 - val_loss: 15.8042
Epoch 6/150
90/90 - 0s - loss: 15.4441 - val_loss: 15.4556
Epoch 7/150
90/90 - 0s - loss: 15.1148 - val_loss: 15.1129
Epoch 8/150
90/90 - 0s - loss: 14.7811 - val_loss: 14.8330
Epoch 9/150
90/90 - 0s - loss: 14.5178 - val_loss: 14.5889
Epoch 10/150
90/90 - 0s - loss: 14.2946 - val_loss: 14.3862
Epoch 11/150
90/90 - 0s - loss: 14.0965 - val_loss: 14.1994
Epoch 12/150
90/90 - 0s - loss: 13.9123 - val_loss: 14.0241
Epoch 13/150
90/90 - 0s - loss: 13.7751 - val_loss: 13.9090
Epoch 14/150
90/90 - 0s - loss: 13.6394 - val_loss: 13.7659
Epoch 15/150
90/90 - 0s - loss: 13.5291 - val_loss: 13.6474
Epoch 16/150
90/90 - 0s - loss: 13.4050 - val_loss: 13.5433
Epoch 17/150
90/90 - 0s - loss: 13.3033 - val_los

In [ ]:
# make a prediction
yhat = model.predict(test_X)
test_X = test_X.reshape((test_X.shape[0], n_hours*n_features))

# invert scaling for forecast (não foi feito)
inv_yhat = concatenate((yhat, test_X[:, -9:]), axis=1)
inv_yhat = inv_yhat[:,0]

# invert scaling for actual
test_y = test_y.reshape((len(test_y), 1))
inv_y = concatenate((test_y, test_X[:, -9:]), axis=1)
# inv_y = scaler.inverse_transform(inv_y)
inv_y = inv_y[:,0]

# calculate RMSE
rmse = sqrt(mean_squared_error(inv_y, inv_yhat))
print('Test RMSE: %.3f' % rmse)

Test RMSE: 16.950


In [ ]:
# calculate MAE
mae = mean_absolute_error(inv_y, inv_yhat)
print('Test MAE: %.3f' % mae)

Test MAE: 11.380


In [ ]:
# calculate R2
r2=r2_score(inv_y, inv_yhat)
print('Test R2: %.3f' % r2)

Test R2: 0.498


#**(mask) janela=1**

In [ ]:
df_sc_bruto=df_sc_2
df_sc_bruto.drop(columns=['Ano','Mês','Dia','Hora','Hora-minuto'], inplace=True)
df_sc_bruto_copy=df_sc_bruto.copy()
df_sc_bruto_copy=df_sc_bruto_copy[['O3']]
df_sc_bruto.drop(columns=['O3'],inplace=True)
df_sc_bruto = pd.concat([df_sc_bruto_copy,df_sc_bruto], axis=1, ignore_index=False)

# prepare data for lstm
# prepare data for lstm
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score #colocar citação
from keras.layers import Masking

from math import sqrt
from numpy import concatenate
from matplotlib import pyplot

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.optimizers import Adam
import numpy as np

values = df_sc_bruto.values
# scaler = MinMaxScaler(feature_range=(0, 1))
# scaled = scaler.fit_transform(df_sc_bruto)
scaled=values
df=pd.DataFrame(scaled)
df = df.replace(np.NaN,-9999)
values = df.values

# convert series to supervised learning
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
	n_vars = 1 if type(data) is list else data.shape[1]
	df = DataFrame(data)
	cols, names = list(), list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
		names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
		if i == 0:
			names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
		else:
			names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
	# put it all together
	agg = concat(cols, axis=1)
	agg.columns = names
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg

# specify the number of lag hours
n_hours = 1
n_features = 10

# frame as supervised learning
reframed = series_to_supervised(scaled, n_hours, 1) #janela

# split into train and test sets
values = reframed.values
n_train_hours = 1855 * 24
train = values[:n_train_hours, :]
test = values[n_train_hours:, :]

# split into input and outputs
n_obs = n_hours * n_features
train_X, train_y = train[:, :n_obs], train[:, -n_features]
test_X, test_y = test[:, :n_obs], test[:, -n_features]

# reshape input to be 3D [samples, timesteps, features]
train_X = train_X.reshape((train_X.shape[0], n_hours, n_features))
test_X = test_X.reshape((test_X.shape[0], n_hours, n_features))

# design network
opt = Adam(learning_rate=0.0005)
model = Sequential()
model.add(Masking(mask_value=-9999, input_shape=(train_X.shape[1], train_X.shape[2])))
model.add(LSTM(300))
model.add(Dense(1))
model.compile(loss='mae', optimizer=opt, sample_weight_mode='temporal')

# fit network
history = model.fit(train_X, train_y, epochs=150, batch_size=500, validation_data=(test_X, test_y), verbose=2, shuffle=False)

Epoch 1/150
90/90 - 2s - loss: 19.1054 - val_loss: 23.6370
Epoch 2/150
90/90 - 1s - loss: 17.0125 - val_loss: 21.6635
Epoch 3/150
90/90 - 1s - loss: 15.5304 - val_loss: 20.0050
Epoch 4/150
90/90 - 1s - loss: 14.2512 - val_loss: 18.5535
Epoch 5/150
90/90 - 1s - loss: 13.1329 - val_loss: 17.2487
Epoch 6/150
90/90 - 1s - loss: 12.2315 - val_loss: 16.2223
Epoch 7/150
90/90 - 1s - loss: 11.4601 - val_loss: 15.2963
Epoch 8/150
90/90 - 1s - loss: 10.8016 - val_loss: 14.4880
Epoch 9/150
90/90 - 1s - loss: 10.2670 - val_loss: 13.7827
Epoch 10/150
90/90 - 1s - loss: 9.8056 - val_loss: 13.1900
Epoch 11/150
90/90 - 1s - loss: 9.4007 - val_loss: 12.7047
Epoch 12/150
90/90 - 1s - loss: 9.0418 - val_loss: 12.2022
Epoch 13/150
90/90 - 1s - loss: 8.7495 - val_loss: 11.8078
Epoch 14/150
90/90 - 1s - loss: 8.4950 - val_loss: 11.4624
Epoch 15/150
90/90 - 1s - loss: 8.2502 - val_loss: 11.1106
Epoch 16/150
90/90 - 1s - loss: 8.0518 - val_loss: 10.8646
Epoch 17/150
90/90 - 1s - loss: 7.8776 - val_loss: 10.63

In [ ]:
# make a prediction
yhat = model.predict(test_X)
test_X = test_X.reshape((test_X.shape[0], n_hours*n_features))

# invert scaling for forecast (não foi feito)
inv_yhat = concatenate((yhat, test_X[:, -9:]), axis=1)
inv_yhat = inv_yhat[:,0]

# invert scaling for actual
test_y = test_y.reshape((len(test_y), 1))
inv_y = concatenate((test_y, test_X[:, -9:]), axis=1)
# inv_y = scaler.inverse_transform(inv_y)
inv_y = inv_y[:,0]

# calculate RMSE
rmse = sqrt(mean_squared_error(inv_y, inv_yhat))
print('Test RMSE: %.3f' % rmse)

Test RMSE: 12.310


In [ ]:
# calculate MAE
mae = mean_absolute_error(inv_y, inv_yhat)
print('Test MAE: %.3f' % mae)

Test MAE: 7.474


In [ ]:
# calculate R2
r2=r2_score(inv_y, inv_yhat)
print('Test R2: %.3f' % r2)

Test R2: 0.816


#**(mask) janela=2**

In [ ]:
df_sc_bruto=df_sc_2
df_sc_bruto.drop(columns=['Ano','Mês','Dia','Hora','Hora-minuto'], inplace=True)
df_sc_bruto_copy=df_sc_bruto.copy()
df_sc_bruto_copy=df_sc_bruto_copy[['O3']]
df_sc_bruto.drop(columns=['O3'],inplace=True)
df_sc_bruto = pd.concat([df_sc_bruto_copy,df_sc_bruto], axis=1, ignore_index=False)

# prepare data for lstm
# prepare data for lstm
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score #colocar citação
from keras.layers import Masking

from math import sqrt
from numpy import concatenate
from matplotlib import pyplot

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.optimizers import Adam
import numpy as np

values = df_sc_bruto.values
# scaler = MinMaxScaler(feature_range=(0, 1))
# scaled = scaler.fit_transform(df_sc_bruto)
scaled=values
df=pd.DataFrame(scaled)
df = df.replace(np.NaN,-9999)
values = df.values

# convert series to supervised learning
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
	n_vars = 1 if type(data) is list else data.shape[1]
	df = DataFrame(data)
	cols, names = list(), list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
		names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
		if i == 0:
			names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
		else:
			names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
	# put it all together
	agg = concat(cols, axis=1)
	agg.columns = names
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg

# specify the number of lag hours
n_hours = 2
n_features = 10

# frame as supervised learning
reframed = series_to_supervised(scaled, n_hours, 2) #janela

# split into train and test sets
values = reframed.values
n_train_hours = 1855 * 24
train = values[:n_train_hours, :]
test = values[n_train_hours:, :]

# split into input and outputs
n_obs = n_hours * n_features
train_X, train_y = train[:, :n_obs], train[:, -n_features]
test_X, test_y = test[:, :n_obs], test[:, -n_features]

# reshape input to be 3D [samples, timesteps, features]
train_X = train_X.reshape((train_X.shape[0], n_hours, n_features))
test_X = test_X.reshape((test_X.shape[0], n_hours, n_features))

# design network
opt = Adam(learning_rate=0.0005)
model = Sequential()
model.add(Masking(mask_value=-9999, input_shape=(train_X.shape[1], train_X.shape[2])))
model.add(LSTM(300))
model.add(Dense(1))
model.compile(loss='mae', optimizer=opt, sample_weight_mode='temporal')

# fit network
history = model.fit(train_X, train_y, epochs=150, batch_size=500, validation_data=(test_X, test_y), verbose=2, shuffle=False)

Epoch 1/150
90/90 - 2s - loss: 19.3857 - val_loss: 25.2930
Epoch 2/150
90/90 - 1s - loss: 16.7847 - val_loss: 22.9869
Epoch 3/150
90/90 - 1s - loss: 15.4236 - val_loss: 21.4638
Epoch 4/150
90/90 - 1s - loss: 14.3298 - val_loss: 20.0416
Epoch 5/150
90/90 - 1s - loss: 13.4321 - val_loss: 18.9668
Epoch 6/150
90/90 - 1s - loss: 12.7176 - val_loss: 18.0733
Epoch 7/150
90/90 - 1s - loss: 12.1053 - val_loss: 17.2617
Epoch 8/150
90/90 - 1s - loss: 11.6211 - val_loss: 16.7186
Epoch 9/150
90/90 - 1s - loss: 11.1818 - val_loss: 16.1021
Epoch 10/150
90/90 - 1s - loss: 10.8732 - val_loss: 15.6428
Epoch 11/150
90/90 - 1s - loss: 10.5903 - val_loss: 15.2283
Epoch 12/150
90/90 - 1s - loss: 10.3580 - val_loss: 14.8993
Epoch 13/150
90/90 - 1s - loss: 10.1688 - val_loss: 14.5684
Epoch 14/150
90/90 - 1s - loss: 9.9879 - val_loss: 14.3331
Epoch 15/150
90/90 - 1s - loss: 9.8634 - val_loss: 14.1320
Epoch 16/150
90/90 - 1s - loss: 9.7201 - val_loss: 13.9439
Epoch 17/150
90/90 - 1s - loss: 9.6147 - val_loss: 1

In [ ]:
# make a prediction
yhat = model.predict(test_X)
test_X = test_X.reshape((test_X.shape[0], n_hours*n_features))

# invert scaling for forecast (não foi feito)
inv_yhat = concatenate((yhat, test_X[:, -9:]), axis=1)
inv_yhat = inv_yhat[:,0]

# invert scaling for actual
test_y = test_y.reshape((len(test_y), 1))
inv_y = concatenate((test_y, test_X[:, -9:]), axis=1)
# inv_y = scaler.inverse_transform(inv_y)
inv_y = inv_y[:,0]

# calculate RMSE
rmse = sqrt(mean_squared_error(inv_y, inv_yhat))
print('Test RMSE: %.3f' % rmse)

Test RMSE: 18.340


In [ ]:
# calculate MAE
mae = mean_absolute_error(inv_y, inv_yhat)
print('Test MAE: %.3f' % mae)

Test MAE: 11.687


In [ ]:
# calculate R2
r2=r2_score(inv_y, inv_yhat)
print('Test R2: %.3f' % r2)

Test R2: 0.636


#**(mask) janela=3**

In [ ]:
df_sc_bruto=df_sc_2
df_sc_bruto.drop(columns=['Ano','Mês','Dia','Hora','Hora-minuto'], inplace=True)
df_sc_bruto_copy=df_sc_bruto.copy()
df_sc_bruto_copy=df_sc_bruto_copy[['O3']]
df_sc_bruto.drop(columns=['O3'],inplace=True)
df_sc_bruto = pd.concat([df_sc_bruto_copy,df_sc_bruto], axis=1, ignore_index=False)

# prepare data for lstm
# prepare data for lstm
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score #colocar citação
from keras.layers import Masking

from math import sqrt
from numpy import concatenate
from matplotlib import pyplot

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.optimizers import Adam
import numpy as np

values = df_sc_bruto.values
# scaler = MinMaxScaler(feature_range=(0, 1))
# scaled = scaler.fit_transform(df_sc_bruto)
scaled=values
df=pd.DataFrame(scaled)
df = df.replace(np.NaN,-9999)
values = df.values

# convert series to supervised learning
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
	n_vars = 1 if type(data) is list else data.shape[1]
	df = DataFrame(data)
	cols, names = list(), list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
		names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
		if i == 0:
			names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
		else:
			names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
	# put it all together
	agg = concat(cols, axis=1)
	agg.columns = names
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg

# specify the number of lag hours
n_hours = 3
n_features = 10

# frame as supervised learning
reframed = series_to_supervised(scaled, n_hours, 3) #janela

# split into train and test sets
values = reframed.values
n_train_hours = 1855 * 24
train = values[:n_train_hours, :]
test = values[n_train_hours:, :]

# split into input and outputs
n_obs = n_hours * n_features
train_X, train_y = train[:, :n_obs], train[:, -n_features]
test_X, test_y = test[:, :n_obs], test[:, -n_features]

# reshape input to be 3D [samples, timesteps, features]
train_X = train_X.reshape((train_X.shape[0], n_hours, n_features))
test_X = test_X.reshape((test_X.shape[0], n_hours, n_features))

# design network
opt = Adam(learning_rate=0.0005)
model = Sequential()
model.add(Masking(mask_value=-9999, input_shape=(train_X.shape[1], train_X.shape[2])))
model.add(LSTM(300))
model.add(Dense(1))
model.compile(loss='mae', optimizer=opt, sample_weight_mode='temporal')

# fit network
history = model.fit(train_X, train_y, epochs=150, batch_size=500, validation_data=(test_X, test_y), verbose=2, shuffle=False)

Epoch 1/150
90/90 - 2s - loss: 20.5942 - val_loss: 26.9646
Epoch 2/150
90/90 - 1s - loss: 18.8291 - val_loss: 25.2856
Epoch 3/150
90/90 - 1s - loss: 17.9305 - val_loss: 24.0609
Epoch 4/150
90/90 - 1s - loss: 17.1192 - val_loss: 23.0319
Epoch 5/150
90/90 - 1s - loss: 16.4611 - val_loss: 22.1988
Epoch 6/150
90/90 - 1s - loss: 15.8941 - val_loss: 21.5061
Epoch 7/150
90/90 - 1s - loss: 15.3923 - val_loss: 20.8992
Epoch 8/150
90/90 - 1s - loss: 14.9609 - val_loss: 20.3777
Epoch 9/150
90/90 - 1s - loss: 14.6274 - val_loss: 19.9224
Epoch 10/150
90/90 - 1s - loss: 14.2732 - val_loss: 19.5107
Epoch 11/150
90/90 - 1s - loss: 13.9823 - val_loss: 19.1250
Epoch 12/150
90/90 - 1s - loss: 13.6985 - val_loss: 18.7682
Epoch 13/150
90/90 - 1s - loss: 13.4645 - val_loss: 18.4383
Epoch 14/150
90/90 - 1s - loss: 13.2451 - val_loss: 18.1352
Epoch 15/150
90/90 - 1s - loss: 13.0863 - val_loss: 17.9022
Epoch 16/150
90/90 - 1s - loss: 12.9094 - val_loss: 17.6425
Epoch 17/150
90/90 - 1s - loss: 12.7803 - val_los

In [ ]:
# make a prediction
yhat = model.predict(test_X)
test_X = test_X.reshape((test_X.shape[0], n_hours*n_features))

# invert scaling for forecast (não foi feito)
inv_yhat = concatenate((yhat, test_X[:, -9:]), axis=1)
inv_yhat = inv_yhat[:,0]

# invert scaling for actual
test_y = test_y.reshape((len(test_y), 1))
inv_y = concatenate((test_y, test_X[:, -9:]), axis=1)
# inv_y = scaler.inverse_transform(inv_y)
inv_y = inv_y[:,0]

# calculate RMSE
rmse = sqrt(mean_squared_error(inv_y, inv_yhat))
print('Test RMSE: %.3f' % rmse)

Test RMSE: 21.913


In [ ]:
# calculate MAE
mae = mean_absolute_error(inv_y, inv_yhat)
print('Test MAE: %.3f' % mae)

Test MAE: 14.185


In [ ]:
# calculate R2
r2=r2_score(inv_y, inv_yhat)
print('Test R2: %.3f' % r2)

Test R2: 0.505


#**(mask) janela=4**

In [ ]:
df_sc_bruto=df_sc_2
df_sc_bruto.drop(columns=['Ano','Mês','Dia','Hora','Hora-minuto'], inplace=True)
df_sc_bruto_copy=df_sc_bruto.copy()
df_sc_bruto_copy=df_sc_bruto_copy[['O3']]
df_sc_bruto.drop(columns=['O3'],inplace=True)
df_sc_bruto = pd.concat([df_sc_bruto_copy,df_sc_bruto], axis=1, ignore_index=False)

# prepare data for lstm
# prepare data for lstm
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score #colocar citação
from keras.layers import Masking

from math import sqrt
from numpy import concatenate
from matplotlib import pyplot

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.optimizers import Adam
import numpy as np

values = df_sc_bruto.values
# scaler = MinMaxScaler(feature_range=(0, 1))
# scaled = scaler.fit_transform(df_sc_bruto)
scaled=values
df=pd.DataFrame(scaled)
df = df.replace(np.NaN,-9999)
values = df.values

# convert series to supervised learning
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
	n_vars = 1 if type(data) is list else data.shape[1]
	df = DataFrame(data)
	cols, names = list(), list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
		names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
		if i == 0:
			names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
		else:
			names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
	# put it all together
	agg = concat(cols, axis=1)
	agg.columns = names
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg

# specify the number of lag hours
n_hours = 4
n_features = 10

# frame as supervised learning
reframed = series_to_supervised(scaled, n_hours, 4) #janela

# split into train and test sets
values = reframed.values
n_train_hours = 1855 * 24
train = values[:n_train_hours, :]
test = values[n_train_hours:, :]

# split into input and outputs
n_obs = n_hours * n_features
train_X, train_y = train[:, :n_obs], train[:, -n_features]
test_X, test_y = test[:, :n_obs], test[:, -n_features]

# reshape input to be 3D [samples, timesteps, features]
train_X = train_X.reshape((train_X.shape[0], n_hours, n_features))
test_X = test_X.reshape((test_X.shape[0], n_hours, n_features))

# design network
opt = Adam(learning_rate=0.0005)
model = Sequential()
model.add(Masking(mask_value=-9999, input_shape=(train_X.shape[1], train_X.shape[2])))
model.add(LSTM(300))
model.add(Dense(1))
model.compile(loss='mae', optimizer=opt, sample_weight_mode='temporal')

# fit network
history = model.fit(train_X, train_y, epochs=150, batch_size=500, validation_data=(test_X, test_y), verbose=2, shuffle=False)

Epoch 1/150
90/90 - 2s - loss: 21.8837 - val_loss: 28.2444
Epoch 2/150
90/90 - 1s - loss: 20.0129 - val_loss: 27.3833
Epoch 3/150
90/90 - 1s - loss: 19.6104 - val_loss: 26.8552
Epoch 4/150
90/90 - 1s - loss: 19.0888 - val_loss: 26.3138
Epoch 5/150
90/90 - 1s - loss: 18.6205 - val_loss: 25.7796
Epoch 6/150
90/90 - 1s - loss: 18.1455 - val_loss: 25.2315
Epoch 7/150
90/90 - 1s - loss: 17.7389 - val_loss: 24.6968
Epoch 8/150
90/90 - 1s - loss: 17.4179 - val_loss: 24.2925
Epoch 9/150
90/90 - 1s - loss: 17.1002 - val_loss: 23.9114
Epoch 10/150
90/90 - 1s - loss: 16.7869 - val_loss: 23.5142
Epoch 11/150
90/90 - 1s - loss: 16.5205 - val_loss: 23.2106
Epoch 12/150
90/90 - 1s - loss: 16.3388 - val_loss: 22.9666
Epoch 13/150
90/90 - 1s - loss: 16.1410 - val_loss: 22.7365
Epoch 14/150
90/90 - 1s - loss: 15.9156 - val_loss: 22.4590
Epoch 15/150
90/90 - 1s - loss: 15.7608 - val_loss: 22.2852
Epoch 16/150
90/90 - 1s - loss: 15.6355 - val_loss: 22.1121
Epoch 17/150
90/90 - 1s - loss: 15.4787 - val_los

In [ ]:
# make a prediction
yhat = model.predict(test_X)
test_X = test_X.reshape((test_X.shape[0], n_hours*n_features))

# invert scaling for forecast (não foi feito)
inv_yhat = concatenate((yhat, test_X[:, -9:]), axis=1)
inv_yhat = inv_yhat[:,0]

# invert scaling for actual
test_y = test_y.reshape((len(test_y), 1))
inv_y = concatenate((test_y, test_X[:, -9:]), axis=1)
# inv_y = scaler.inverse_transform(inv_y)
inv_y = inv_y[:,0]

# calculate RMSE
rmse = sqrt(mean_squared_error(inv_y, inv_yhat))
print('Test RMSE: %.3f' % rmse)

Test RMSE: 27.357


In [ ]:
# calculate MAE
mae = mean_absolute_error(inv_y, inv_yhat)
print('Test MAE: %.3f' % mae)

Test MAE: 18.331


In [ ]:
# calculate R2
r2=r2_score(inv_y, inv_yhat)
print('Test R2: %.3f' % r2)

Test R2: 0.417
